# RNN情意分析實作
資料集：imdb電影評論資料集

### 1. 讀入深度學習套件

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.preprocessing import sequence #用來做一連串處理的
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding #embedding可以將輸入的10000維向量做適當的縮小
from tensorflow.keras.layers import GRU
from tensorflow.keras.datasets import imdb

### 2. 讀入數據

一般自然語言處理, 我們會限制最大要使用的字數。

In [ ]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

17473536/17464789 [==============================] - 0s 0us/step


In [ ]:
len(x_train)

25000

In [ ]:
len(x_test)

25000

In [ ]:
len(x_train[0])

218

In [ ]:
len(x_train[1])

189

In [ ]:
y_train[0]

1

In [ ]:
y_train[1]

0

### 3. 資料處理

將最常出現的**120**個字當作我們的輸入資料。

In [ ]:
x_train = sequence.pad_sequences(x_train, maxlen=120)
x_test = sequence.pad_sequences(x_test, maxlen=120)

### 4. step 01: 打造一個函數學習機
改為使用**GRU**作為中間層，並降低神經元個數為**100個**

In [ ]:
model = Sequential()

In [ ]:
model.add(Embedding(10000, 100))

In [ ]:
model.add(GRU(100))

In [ ]:
model.add(Dense(1, activation='sigmoid'))

#### 組裝

In [ ]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

#### 欣賞我們的 model

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 100)         1000000   
                                                                 
 gru_1 (GRU)                 (None, 100)               60600     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 1,060,701
Trainable params: 1,060,701
Non-trainable params: 0
_________________________________________________________________


### 5. step 02: 訓練

改為使用batch_size=50!

達到訓練集96%正確率、驗證集近85%正確率



In [ ]:
model.fit(x_train, y_train, batch_size=50, epochs=5,
         validation_data=(x_test, y_test))

Epoch 1/5
500/500 [==============================] - 24s 44ms/step - loss: 0.4198 - accuracy: 0.7974 - val_loss: 0.3407 - val_accuracy: 0.8537
Epoch 2/5
500/500 [==============================] - 21s 42ms/step - loss: 0.2456 - accuracy: 0.9021 - val_loss: 0.3322 - val_accuracy: 0.8580
Epoch 3/5
500/500 [==============================] - 20s 41ms/step - loss: 0.1677 - accuracy: 0.9386 - val_loss: 0.3912 - val_accuracy: 0.8558
Epoch 4/5
500/500 [==============================] - 21s 42ms/step - loss: 0.1049 - accuracy: 0.9628 - val_loss: 0.4755 - val_accuracy: 0.8448
Epoch 5/5
500/500 [==============================] - 20s 40ms/step - loss: 0.0701 - accuracy: 0.9758 - val_loss: 0.4835 - val_accuracy: 0.8442


### 6. step 03: 測試

In [ ]:
from tensorflow.keras.datasets.imdb import get_word_index

In [ ]:
word_index = get_word_index()

1654784/1641221 [==============================] - 0s 0us/step


In [ ]:
word_index['this']

11

In [ ]:
text = "this movie is worth seeing"

In [ ]:
seq = [word_index[x] for x in text.split()]

In [ ]:
model.predict([seq])

array([[0.9938752]], dtype=float32)

In [ ]:
text = "could of been so much better if properly cast directed and a better script"

In [ ]:
seq = [word_index[x] for x in text.split()]

In [ ]:
model.predict([seq])

array([[0.74226993]], dtype=float32)

### 6. 換個存檔方式

這次是把 model 和訓練權重分開存, 使用上更有彈性。

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/My Drive/Colab Notebooks'

/content/drive/My Drive/Colab Notebooks


In [ ]:
model_json = model.to_json()
open('imdb_model_architecture.json', 'w').write(model_json)
model.save_weights('ismdb_model_weights.h5')